Importing requried libraries

In [1]:
from groq import Groq
from dotenv import load_dotenv
import os
import pprint
import json
import re

load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")

Taking user input + the research context [Research context needs to be improved or reimplimented]

In [2]:
#----------Cell 2----------


user_input = "research on iphones"
research_context = "Starting fresh research"

Defining the necessary paths where the data needs to be stored for the other models to access

In [3]:
#----------Paths----------

task_data_path = "../../model_output_data/"


In [4]:
#-------Cleaning Input For Folder Name-------

lower_user_input = user_input.lower()

# Matches any character NOT in the range a-z, A-Z, or 0-9
cleaned_text = re.sub(r'[^a-zA-Z0-9\s]+', '', lower_user_input)

folder_name_for_query = re.sub(r"\s+", "_", cleaned_text)

print(folder_name_for_query)

research_on_iphones


In [5]:

complete_data_path_query = task_data_path + folder_name_for_query

print(complete_data_path_query)
if not os.path.exists(complete_data_path_query):
    os.makedirs(complete_data_path_query, exist_ok=True)
    print(f"Created folder: {complete_data_path_query}")
else:
    print(f"Folder already exists: {complete_data_path_query}")

../../model_output_data/research_on_iphones
Created folder: ../../model_output_data/research_on_iphones


Temporary Storage List to ensure the LLM remembers the previous conversation with the user

[This could possibly be removed and be implimented in further model as the user won't really interact with this specific agent]

In [6]:
messages = []
# MAX_MESSAGES = 10   # total, user + assistant



# if len(messages) > MAX_MESSAGES:
#     messages = messages[-MAX_MESSAGES:]




The system prompt that tells the LLM what it's actual purpose is so it doesn't get out of track

In [7]:
system_prompt = f"""
You are creating an initial research plan for the topic: "{user_input}"
Initial Query: "{user_input}"
Research Context: {research_context if research_context else "Starting fresh research"}
Decompose this query into 3–5 actionable research tasks. Return a JSON array with each task having:
• "description": Clear, actionable task (string)
• "priority": 1–10 (integer, higher = more important, default=5)
• "type": "research" (always "research")

STRICTLY CREATE ONLY 3-5 ACTIONABLE RESEARCH TASKS

Focus on: understanding the topic, gathering information, identifying key aspects, and building foundational knowledge.
Example for "Impacts of Generative AI on Scientific Research":
<answer>
[
{{"description": "Survey major applications of generative AI in scientific discovery", "priority": 8, "type": "research"}},
{{"description": "Identify key papers and institutions leading AI-assisted science research", "priority": 7, "type": "research"}},
{{"description": "Examine methodological advances enabled by generative models in ...", "priority": 6, "type": "research"}},
{{"description": "Assess challenges and ethical considerations of AI-generated scientific results", "priority": 5, "type": "research"}}
]
</answer>
CRITICAL: Wrap JSON in <answer>tags.
Output ONLY valid JSON.


"""

messages.append({"role": "system", "content": system_prompt})

First Agent by groq, the task agent: Responsible for creating tasks for the user query for the next model to find information for the tasks

Model Used: llama-3.1-8b-instant
Why ? Faster Responses, bigger context window and great in text generation



In [8]:
messages.append({"role": "user", "content": user_input}) 

task_agent = Groq()
completion = task_agent.chat.completions.create(
    model="llama-3.1-8b-instant",
    messages=messages,
    stream=False
)

reply = completion.choices[0].message.content
print(reply)
messages.append({"role": "assistant", "content": reply})

<answer>
[
  {"description": "Conduct a literature review on the history of iPhone development, from the first iPhone in 2007 to the latest models", "priority": 6, "type": "research"},
  {"description": "Identify and analyze key iPhone features and technologies (e.g., Touch ID, Face ID, camera advancements)", "priority": 7, "type": "research"},
  {"description": "Study market trends and sales data of iPhone models to understand consumer preferences and market share ", "priority": 5, "type": "research"},
  {"description": "Explore the impact of iPhones on mobile software development, including apps, games, and services", "priority": 8, "type": "research"},
  {"description": "Research the environmental and social implications of iPhone production and usage (e.g., e-waste, supply chains, labor practices)", "priority": 4, "type": "research"}
]
</answer>


In [9]:
print(messages[-1]['content'])

<answer>
[
  {"description": "Conduct a literature review on the history of iPhone development, from the first iPhone in 2007 to the latest models", "priority": 6, "type": "research"},
  {"description": "Identify and analyze key iPhone features and technologies (e.g., Touch ID, Face ID, camera advancements)", "priority": 7, "type": "research"},
  {"description": "Study market trends and sales data of iPhone models to understand consumer preferences and market share ", "priority": 5, "type": "research"},
  {"description": "Explore the impact of iPhones on mobile software development, including apps, games, and services", "priority": 8, "type": "research"},
  {"description": "Research the environmental and social implications of iPhone production and usage (e.g., e-waste, supply chains, labor practices)", "priority": 4, "type": "research"}
]
</answer>


In [10]:
assistant_output = messages[-1]["content"]

Appending the generated tasks to a json file for the other model to make use of the generated tasks

In [11]:
start = assistant_output.find("<answer>") + len("<answer>")
end = assistant_output.find("</answer>")

json_text = assistant_output[start:end].strip()

tasks = json.loads(json_text)

with open(os.path.join(complete_data_path_query, "tasks.json"), "w") as f:
    json.dump(tasks, f, indent=4)

print(tasks)

[{'description': 'Conduct a literature review on the history of iPhone development, from the first iPhone in 2007 to the latest models', 'priority': 6, 'type': 'research'}, {'description': 'Identify and analyze key iPhone features and technologies (e.g., Touch ID, Face ID, camera advancements)', 'priority': 7, 'type': 'research'}, {'description': 'Study market trends and sales data of iPhone models to understand consumer preferences and market share ', 'priority': 5, 'type': 'research'}, {'description': 'Explore the impact of iPhones on mobile software development, including apps, games, and services', 'priority': 8, 'type': 'research'}, {'description': 'Research the environmental and social implications of iPhone production and usage (e.g., e-waste, supply chains, labor practices)', 'priority': 4, 'type': 'research'}]
